### Homework 8

이번 과제에서는 이분산 모형의 추정과 리스크 관리와 관련된 고급 통계 기법들을 실습해보도록 한다

#### 1. Data Import

yfinance library를 이용하여 2020년 1월 1일부터 2023년 7월 31일까지의 IBM 기업의 주가를 import 하여 log defference 형태로 수정하라

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

#### 2. GARCH models

**2.1 GARCH model**

$\mathrm{GARCH}$ 모형은 다음과 같이 추정된다

$$r_t = \mu + a_t$$
$$a_t = \sigma_t \epsilon_t$$
$$\sigma_t^2 = \alpha_0 + \sum_{i=1}^m \alpha_i a_{t-1}^2 + \sum_{j=1}^s \beta_j \sigma_{t-j}^2$$

IBM 수익률 벡터를 활용하여 적절한 order를 선택해 $\mathrm{GARCH}$ model을 추정하고, 추정된 변동성을 plotting하여라

In [ ]:
import arch

**2.2 GARCH-Mean model**

$\mathrm{GARCH}(1,1)\mathrm{-M}$모형은 다음과 같이 쓸 수 있다

$$r_t = \mu + c \sigma_t^2 + a_t$$
$$a_t = \sigma_t \epsilon_t$$
$$\sigma_t^2 = \alpha_0 + \alpha_1a_{t-1}^2 + \beta_1 \sigma_{t-1}^2$$

이는 $r_t$에 대해 상수항이 있다고 가정하여 추정하는 모형이다. 한편, python에서는 arch library의 arch_model 함수에서 parameter `mean = Constant`로 지정하여 추정할 수 있다

IBM 수익률 벡터에 대해서 $\mathrm{GARCH}(1,1)\mathrm{-M}$모형을 추정하라. 추정된 변동성에 대해 적절한 방법으로 plotting하여라. **2.1번**의 추정된 결과와 어떤 차이가 있는가?. IBM 주가에 대해서 $\mathrm{GARCH}(1,1)\mathrm{-M}$ 모형은 유의한가?

**2.3 Exponential GARCH model**

$\mathrm{GARCH}$모형이 금융 시계열에 쓰였을 때 나타나는 단점을 최소화하기 위해서 exponential $\mathrm{GARCH(EGARCH)}$모형을 제안하였다. 좋은 뉴스와 나쁜 뉴스 즉, 음의 충격과 양의 충격에 비대칭적 효과를 고려하여 weight를 조정하였다

$\mathrm{EGARCH}$ 모형은 다음과 같이 나타낼 수 있다

$$r_t = \mu + a_t$$
$$a_t = \sigma_t \epsilon_t$$
$$\ln(\sigma_t^2) = \alpha_0 + \frac{1 +\beta_1 B + \cdots + \beta_{s-1}B^{s-1}}{1 - \alpha_1B - \cdots - \alpha_m B^m}g(\epsilon_{t-1})$$

$$g(\epsilon_t) = \begin{cases}
(\theta + \gamma) \epsilon_t - \gamma E(|\epsilon_t|) ~~~~~ & \mbox{if } \epsilon_t \geq 0\\
(\theta - \gamma) \epsilon_t - \gamma E(|\epsilon_t|) ~~~~~ & \mbox{if } \epsilon_t < 0
\end{cases}$$

한편, python에서는 arch library의 arch_model 함수에서 parameter `vol = EGARCH`로 지정하여 추정할 수 있다

IBM 수익률 벡터에 대해서 $\mathrm{EGARCH}(1,1)$모형을 추정하라. 추정된 변동성에 대해 적절한 방법으로 plotting하여라. **2.2번**의 추정된 결과와 어떤 차이가 있는가?. IBM 주가에 대해서 $\mathrm{EGARCH}(1,1)$ 모형은 유의한가?

#### 3. Value at Risk

재무회계에 있어서 위험은 크게 시장위험, 유동성위험, 신용위험, 운영위험으로 나뉘며, 이중 시장위험을 측정하는데 보편적으로 사용하는 모형은 Value at Risk$\mathrm{(VaR)}$이다. $\mathrm{VaR}$의 측정방법은 다음 네 가지로 분류할 수 있다
1. Econometrics Approach
2. Risk Metrics
3. Quantile Estimation
4. Extreme Value Approach

Econometrics Approach는 $\mathrm{GARCH}$등 변동성 추정 방법을 통해 구하는 방법이며, 다음과 같이 추정한다

$$\mathrm{VaR} = \sigma_t \times z_a$$

여기서 $\sigma_t$는 $\mathrm{GARCH}$모형을 통해 추정된 변동성이며, $z_a$는 한쪽꼬리검정의 신뢰상수를 의미한다. 예를들어, 95%의 신뢰상수는 1.645이며, 99%의 신뢰상수는 2.33이다

**Example 2**에서 추정한 세 가지 종류의 $\mathrm{GARCH}$모형에 대해 신뢰수준 95%에서의 일별 $\mathrm{VaR}$을 추정하라. 세 모형의 $\mathrm{VaR}$ 추정값에는 어떤 차이가 있는가?

#### 4. Stochastic Volatility Model

IBM 수익률 벡터에 대해서 Stochastic Volatility Model을 추정하라. 추정을 위해 아래의 함수를 사용하면 된다. $\nu = 0.1$, step size = 10이고 random seed = 42로 지정하여라

**4.1 Monte Carlo Simulation**

prior distribution은 500개를 추정하였을 때, monte carlo simulation으로 추정된 IBM return의 확률보행 경로를 출력하라

In [ ]:
import os
import arviz as az
import pymc as pm

def StochasticVolatilityModel(data, stepsize : int , nu : float):
    with pm.Model(coords = {"time": data.index.values}) as model:
        step_size = pm.Exponential("step_size", stepsize)
        volatility = pm.GaussianRandomWalk("volatility", sigma = step_size, dims = "time")
        nu = pm.Exponential("nu", nu)
        returns = pm.StudentT("returns", nu = nu, lam = np.exp(-2 * volatility), observed = data, dims = "time")
    return model

**4.2 SV model**

Stochastic Volatility 모형을 이용하여 변동성을 추정하고 plotting하여라

**4.3 GARCH and VaR**

Exercise 2에서 추정한 $\mathrm{GARCH}$모형과 비교하라. $\mathrm{GARCH}$으로 추정한 $\mathrm{VaR}$과 SV model로 추정한 $\mathrm{VaR}$을 비교하라. 차이가 있는가? 있다면 그 이유는 무엇인가? 단, 신뢰구간은 95%로 추정한다

(SV model에서 $\mathrm{VaR}$을 추정할 때 중앙값을 사용해도 무관하다)

#### 5. Becker-Parkinson Volatility

IBM 수익률 벡터를 이용하여 Becker-Parkinson Range Volatility를 추정하여라

**5.1 Corwin & Schultz Spread**

IBM 가격 데이터 행렬과 아래 함수를 활용하여 Corwin & Schultz spread를 추정하라

In [ ]:
def getBeta(series : pd.DataFrame, sl : int) :
    h1 = series[['High','Low']].values
    h1 = np.log(h1[:,0]/h1[:,1])**2
    h1 = pd.Series(h1, index = series.index)
    beta = h1.rolling(window = 2).sum()
    beta = beta.rolling(window = sl).mean()
    beta = beta.replace([np.inf, -np.inf], np.nan)
    return beta.dropna()

def getGamma(series : pd.DataFrame) :
    h2 = series['High'].rolling(window=2).max()
    l2 = series['Low'].rolling(window=2).min()
    gamma = np.log(h2.values/l2.values)**2
    gamma = pd.Series(gamma, index = h2.index)
    gamma = gamma.replace([np.inf, -np.inf], np.nan)
    return gamma.dropna()

def getAlpha(beta, gamma) :
    alpha = ((np.sqrt(2 * beta) - np.sqrt(beta)) /(3 - (2 * np.sqrt(2)))) - np.sqrt(gamma / (3 - (2 * np.sqrt(2))))
    alpha[alpha<0] = 0
    return alpha.dropna()

def corwinSchultz(series, sl : int = 1) :
    beta = getBeta(series, sl)
    gamma = getGamma(series)
    alpha = getAlpha(beta, gamma)
    spread = 2 * np.exp(alpha - 1)/(1 + np.exp(alpha))
    startTime = pd.Series(series.index[0 : spread.shape[0]], index = spread.index)
    spread = pd.concat([spread,startTime], axis = 1)
    spread.columns = ['Spread', 'Start_time']
    return spread

def getSigma(beta, gamma) :
    k2 = (8/np.pi)**.5
    den = 3-2*2**.5
    sigma = (2**-.5-1)*beta**.5/(k2*den)
    sigma += (gamma/(k2**2*den))**.5
    sigma[sigma<0] = 0
    return sigma

**5.2 Becker-Parkinson Volatility**

**5.1번**에서 스프레드를 구하는데 활용한 $\alpha, \beta$를 활용하여 Becker-Parkinson range volatility를 추정하라

**5.3 Kalman Filter**

<center><img src = "https://machinelearningspace.com/wp-content/uploads/2020/02/kalman15.png" alt="My Image"></center>

Kalman Filter는 본래 공학에서 측정치와 상태 공간 사이의 예측 오차를 줄여나감으로써, 물체의 위치를 추정하는데 쓰이는 알고리즘 필터이다. 경제학에서는 대개 통계량과 실제수준과의 괴리감을 줄이고 시장의 기대치 혹은 상태공간을 추정하는데 사용되며, 다음과 같은 과정을 거친다

$$v_t = y_t - \mu_{t|t-1}$$
$$V_t = \Sigma_{t|t-1} + \sigma_e^2$$
$$K_t = \frac{\Sigma_{t|t-1}}{V_t}$$
$$\mu_{t+1|t} = \mu_{t|t-1} + K_t v_t$$
$$\Sigma_{t+1|t} = \Sigma_{t|t-1}(1 - K_t) + \sigma_\eta^2$$

$K_t$는 Kalman Filter 과정에서 얻는 효과이며, Kalman Gain이라고 한다. Kalman Filter를 통해 측정치의 잠재 상태인 **상태공간(State Space)** 을 추정할 수 있으며, 평활화 및 노이즈를 제거할 때에도 사용되고, 최근에는 Machine Learning의 Feature Preprocessing 과정에 주로 사용된다

**5.2번**에서 추정한 Becker-Parkinson Range Volatility에 Kalman Filter를 적용하면 시장의 내재 변동성을 추정할 수 있다. 아래의 Class를 활용하여 IBM return 벡터의 Becker-Parkinson 내재 변동성을 추정하고 $\mathrm{GARCH}$모형의 변동성과 비교하라. 어떤 차이가 존재하는가?

In [ ]:
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
import statsmodels.api as sm
class LocalLinearTrend(sm.tsa.statespace.MLEModel):
    def __init__(self, endog):
        # Model order
        k_states = k_posdef = 2
        # Initialize the statespace
        super(LocalLinearTrend, self).__init__(
            endog, k_states=k_states, k_posdef=k_posdef,
            initialization="approximate_diffuse",
            loglikelihood_burn=k_states
        )
        # Initialize the matrices
        self.ssm['design'] = np.array([1, 0])
        self.ssm['transition'] = np.array([[1, 1],
                                           [0, 1]])
        self.ssm['selection'] = np.eye(k_states)
        # Cache some indices
        self._state_cov_idx = ("state_cov",) + np.diag_indices(k_posdef)

    @property
    def param_names(self):
        return ["sigma2.measurement", "sigma2.level", "sigma2.trend"]

    @property
    def start_params(self):
        return [np.std(self.endog)]*3

    def transform_params(self, unconstrained):
        return unconstrained ** 2

    def untransform_params(self, constrained):
        return constrained ** 0.5

    def update(self, params, *args, **kwargs):
        params = super(LocalLinearTrend, self).update(params, *args, **kwargs)
        # Observation covariance
        self.ssm['obs_cov',0,0] = params[0]
        # State covariance
        self.ssm[self._state_cov_idx] = params[1:]

#### 6. Exponential Weighted oving Average Volatility

변동성을 추정하는 방법은 여러가지가 있지만 그중 지수가중이동평균 변동성(EWMA Volatility)는 계산이 편하고 빠르다는 장점 덕분에 근사값으로 널리 사용되고 있다. 또한, 동적으로 변동성을 추정할 수 있다는 장점도 가지고 있다

EWMA 변동성은 다음과 같이 계산된다

$$\mathrm{EWMA}_t = \lambda r_t + (1-\lambda)\mathrm{EWMA}_{t-1}$$

아래 함수는 일별 변동성을 가중이동평균하여 구하는 함수이다. `span`은 가중이동평균할 기간을 의미하며, 보통 10일을 기준으로 한다

IBM 수익률 시계열에 대하여 10일 가중이동평균 변동성을 구하고 적절한 방법으로 시각화하라. **2.1번**에서 추정한 $\mathrm{GARCH}$ 변동성과 비교하라. EWMA 변동성은 $\mathrm{GARCH}$의 추정치 대신 사용 가능한 지표인가?

In [ ]:
def getDailyVolatility(close : pd.Series, span : int = 10):
    df0 = close.index.searchsorted(close.index - pd.Timedelta(days=1))
    df0 = df0[df0 > 0]
    df0 = (pd.Series(close.index[df0 - 1],
                     index=close.index[close.shape[0] - df0.shape[0]:]))
    try:
        df0 = close.loc[df0.index] / close.loc[df0.values].values - 1  
    except Exception as e:
        print(f'error: {e}\nplease confirm no duplicate indices')
    df0 = df0.ewm(span = span).std().rename('dailyVol')
    return df0